### filesystem directory structure

```
|- project_a
|  |- artifacts
|  |  |- id_a
|  |  |  |- data
|  |  |  |_ metadata.json
|  |  |_ ...
|  |- dataframes
|  |  |- id_a
|  |  |  |- data
|  |  |  |_ metadata.json
|  |  |_ ...
|  |- experiments
|     |- id_a
|     |  |- artifacts
|     |  |  |- id_a
|     |  |  |  |- data
|     |  |  |  |_ metadata.json
|     |  |  |_ ...
|     |  |- dataframes
|     |  |  |- id_a
|     |  |  |  |- data
|     |  |  |  |_ metadata.json
|     |  |  |_ ...
|     |  |- features
|     |  |  |- name_a
|     |  |  |  |_ metadata.json
|     |  |  |_ ...
|     |  |- metrics
|     |  |  |- name_a
|     |  |  |  |_ metadata.json
|     |  |  |_ ...
|     |  |- parameters
|     |  |  |- name_a
|     |  |  |  |_ metadata.json
|     |  |  |_ ...
|     |  |_ metadata.json
|     |- id_b
|     |  |_ ...
|     |_ ...
|- project_b
|  |_ ...
|_ ...
```

### equivalent JSON

```json
{
    "project_a": {
        "artifact": {
            "56387f08-00c0-4bf7-9180-04f414c60991": {
                "id": "56387f08-00c0-4bf7-9180-04f414c60991",
                "parent_id": "c9674165-07bb-439f-b5c4-ea85d5bb3729",
                "created_at": "00:00",
                "data": "binary data",
                "name": "artifact name",
                "tags": ["tag_a", "tag_b"],
            },
            "bfdec5ea-b9ea-4205-8eec-35b8a691346c": {
                ...
            },
            ...
        },
        "dataframe": {},
        "experiment": {},
    },
    "project_b": {
        ...
    },
    ...
}
```

In [1]:
import json
import sys

json.dump(
    {
        "project_a": {
            "artifact": {
                "56387f08-00c0-4bf7-9180-04f414c60991": {
                    "id": "56387f08-00c0-4bf7-9180-04f414c60991",
                    "parent_id": "c9674165-07bb-439f-b5c4-ea85d5bb3729",
                    "created_at": "00:00",
                    "data": "binary data",
                    "name": "artifact name",
                    "tags": ["tag_a", "tag_b"],
                },
                "bfdec5ea-b9ea-4205-8eec-35b8a691346c": {},
            },
            "dataframe": {},
            "experiment": {},
        },
        "project_b": {},
    },
    sys.stdout,
)

{"project_a": {"artifact": {"56387f08-00c0-4bf7-9180-04f414c60991": {"id": "56387f08-00c0-4bf7-9180-04f414c60991", "parent_id": "c9674165-07bb-439f-b5c4-ea85d5bb3729", "created_at": "00:00", "data": "binary data", "name": "artifact name", "tags": ["tag_a", "tag_b"]}, "bfdec5ea-b9ea-4205-8eec-35b8a691346c": {}}, "dataframe": {}, "experiment": {}}, "project_b": {}}